In [1]:
import gym
from agent import bipedal_walker_agent
from Qnetwork import Qnet
from metrics import MetricLogger
from tqdm.auto import tqdm,trange
from pathlib import Path
import datetime

In [2]:
env = gym.make('BipedalWalker-v3')
state = env.reset()
net = Qnet(state_dim=env.observation_space.shape[0], action_dim=env.action_space.shape[0]).float().to("cuda")

In [3]:
save_dir = Path('checkpoints') / datetime.datetime.now().strftime('%Y-%m-%dT%H-%M-%S')
save_dir.mkdir(parents=True)

checkpoint = None 

In [4]:
walker = bipedal_walker_agent(state_dim=env.observation_space.shape[0], action_dim=env.action_space.shape[0], save_dir=save_dir)

Agent initialized


In [7]:
episodes = 1000

with trange(episodes) as t:
    for episode in t:
        state = env.reset()
        done = False
        score = 0
        while not done:
            action = walker.step(state)
            n_state,reward,done,info = env.step(action)
            walker.replay_buffer_save(state, action, reward, n_state, done)
            walker.update()
            score+=reward
            state = n_state
        t.set_postfix(str = f"Steps:{walker.curr_step} Episode: {episode} Score: {score}")
    env.close()  

  0%|          | 0/1000 [00:00<?, ?it/s]

In [8]:
walker.save()

Agent saved to checkpoints/2022-04-30T02-45-08/agent_0.chkpt at step 218118
